In [1]:
from sklearn import tree, preprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn import model_selection
from sklearn import metrics
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier


dataset_teaching = pd.read_csv('teachingAssistantEvaluation.data', sep=',')
print("\nDataset teaching Assistant Evaluation:")
dataset_teaching


Dataset teaching Assistant Evaluation:


,nativeEnglishSpeaker,courseInstructor,course,summerOrRegularSemester,classSize,resultado
0,1,23,3,1,19,3
1,2,15,3,1,17,3
2,1,23,3,2,49,3
3,1,5,2,2,33,3
4,2,7,11,2,55,3
...,...,...,...,...,...,...
146,2,3,2,2,26,1
147,2,10,3,2,12,1
148,1,18,7,2,48,1
149,2,22,1,2,51,1


In [2]:
dataset_teaching.columns

Index(['nativeEnglishSpeaker', 'courseInstructor', 'course',
       'summerOrRegularSemester', 'classSize', 'resultado'],
      dtype='object')

In [3]:
teaching_normalizada = dataset_teaching.values.copy()
normalizador =  StandardScaler()

teaching_normalizada = normalizador.fit_transform(dataset_teaching)
dataset_teaching['courseInstructor'] = teaching_normalizada[:,1]
dataset_teaching['course'] = teaching_normalizada[:,2]
dataset_teaching['classSize'] = teaching_normalizada[:,4]

print("\nDataset Teaching Assistant Evaluation normalizada:")
dataset_teaching


Dataset Teaching Assistant Evaluation normalizada:


,nativeEnglishSpeaker,courseInstructor,course,summerOrRegularSemester,classSize,resultado
0,1,1.375485,-0.729359,1,-0.690028,3
1,2,0.199557,-0.729359,1,-0.845658,3
2,1,1.375485,-0.729359,2,1.644422,3
3,1,-1.270352,-0.872203,2,0.399382,3
4,2,-0.976370,0.413398,2,2.111312,3
...,...,...,...,...,...,...
146,2,-1.564334,-0.872203,2,-0.145323,1
147,2,-0.535398,-0.729359,2,-1.234733,1
148,1,0.640530,-0.157980,2,1.566607,1
149,2,1.228494,-1.015048,2,1.800052,1


In [4]:
teaching_values = dataset_teaching.iloc[:,0:6].values
print("\nTeaching Assistant Evaluation features:\n")
print(teaching_values)


Teaching Assistant Evaluation features:

[[ 1.          1.37548494 -0.7293586   1.         -0.69002837  3.        ]
 [ 2.          0.19955726 -0.7293586   1.         -0.84565836  3.        ]
 [ 1.          1.37548494 -0.7293586   2.          1.6444216   3.        ]
 [ 1.         -1.27035233 -0.87220315  2.          0.39938162  3.        ]
 [ 2.         -0.97637041  0.4133978   2.          2.11131159  3.        ]
 [ 2.          1.37548494 -0.7293586   1.         -0.61221337  3.        ]
 [ 2.         -0.68238849 -0.4436695   2.         -0.69002837  3.        ]
 [ 2.         -0.53539753 -0.7293586   2.         -0.06750838  3.        ]
 [ 1.          1.22849398 -0.7293586   1.          2.34475658  3.        ]
 [ 2.          0.19955726 -0.7293586   1.         -0.61221337  3.        ]
 [ 2.         -0.53539753  1.98468785  2.         -1.46817835  3.        ]
 [ 2.         -0.09442466 -1.0150477   2.          0.16593662  3.        ]
 [ 2.          0.64053014  1.8418433   2.          0.08812

In [5]:
teaching_classes = dataset_teaching.iloc[:,5].values
print("\nTeaching Assistant Evaluation features classes:\n")
print(teaching_classes)
print("\nTeaching Assistant Evaluation features classes shape:")
print(teaching_classes.shape)


Teaching Assistant Evaluation features classes:

[3 3 3 3 3 3 3 3 3 3 3 3 3 3 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1 1 1
 1 1 3 3 3 3 3 3 3 3 3 3 3 3 3 3 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1
 1 1 1 1 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 3 3 3 3 3 3 3 2 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1 1 1 1
 1 1 1]

Teaching Assistant Evaluation features classes shape:
(151,)


In [6]:
kfold_treinamento = model_selection.StratifiedKFold(n_splits=5, shuffle=True, random_state=None)
kfold_ajuste_parametros = model_selection.StratifiedKFold(n_splits=3, shuffle=True, random_state=None)

In [7]:
arvore_decisao = DecisionTreeClassifier()
print("====== Iniciando busca pelos melhores parâmetros do algoritmo Arvore de decisao no dataset Teaching ....\n")

param_dist = {'max_depth':list(np.arange(1,10)), 'min_samples_split':list(np.arange(1,10)) ,'criterion':['entropy'], 'splitter':['best','random']}
grid_search = GridSearchCV(arvore_decisao, param_grid=param_dist, cv=kfold_ajuste_parametros, scoring='accuracy', refit=False)
grid_search.fit(teaching_values, teaching_classes)
decisionTreeBestParams = grid_search.best_params_
print("Decision Tree: %s \n\n" % decisionTreeBestParams)

====== Iniciando busca pelos melhores parâmetros do algoritmo Arvore de decisao no dataset Teaching ....

Decision Tree: {'criterion': 'entropy', 'max_depth': 2, 'min_samples_split': 2, 'splitter': 'best'} 




/home/anapedroso/Documentos/IA-desempenho-algoritmos-classificacao/env/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/anapedroso/Documentos/IA-desempenho-algoritmos-classificacao/env/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/anapedroso/Documentos/IA-desempenho-algoritmos-classificacao/env/lib/python3.8/site-packages/sklearn/tree/_classes.py", line 890, in fit
    super().fit(
  File "/home/anapedroso/Documentos/IA-desempenho-algoritmos-classificacao/env/lib/python3.8/site-packages/sklearn/tree/_classes.py", line 228, in fit
    raise ValueError("min_samples_split must be an integer "
ValueError: min_samples_split must be an integer greater than 1 or a float in (0.0, 1.

In [8]:
knn_vizinhos_proximos = KNeighborsClassifier()
print("====== Iniciando busca pelos melhores parâmetros do algoritmo KNN no dataset Teaching ....\n")

param_dist = {'n_neighbors':[3,5,7,9], 'metric':['euclidean', 'manhattan'], 'weights':['uniform', 'distance']} #Parâmetros testados

grid_search = GridSearchCV(knn_vizinhos_proximos, param_grid=param_dist, cv=kfold_ajuste_parametros, scoring='accuracy', refit=False)
grid_search.fit(teaching_values, teaching_classes)
knnBestParams = grid_search.best_params_
print("KNN: %s \n\n" % knnBestParams)

====== Iniciando busca pelos melhores parâmetros do algoritmo KNN no dataset Teaching ....

KNN: {'metric': 'manhattan', 'n_neighbors': 9, 'weights': 'distance'} 




In [9]:
regressao_logistica = LogisticRegression()
print("====== Iniciando busca pelos melhores parâmetros do algoritmo Regressao logistica no dataset Teaching ....\n")

param_dist = {'penalty':['l2','l1'], 'max_iter':[200,300,400,500],'solver':['liblinear','sag','saga']} #Parâmetros testados

grid_search = GridSearchCV(regressao_logistica, param_grid=param_dist, cv=kfold_ajuste_parametros, scoring='accuracy', refit=False)
grid_search.fit(teaching_values, teaching_classes)
regressao_logistica_param = grid_search.best_params_
print("Regressão logistica: %s \n\n" % regressao_logistica_param)

====== Iniciando busca pelos melhores parâmetros do algoritmo Regressao logistica no dataset Teaching ....

Regressão logistica: {'max_iter': 200, 'penalty': 'l2', 'solver': 'sag'} 




/home/anapedroso/Documentos/IA-desempenho-algoritmos-classificacao/env/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/home/anapedroso/Documentos/IA-desempenho-algoritmos-classificacao/env/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/home/anapedroso/Documentos/IA-desempenho-algoritmos-classificacao/env/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/home/anapedroso/Documentos/IA-desempenho-algoritmos-classificacao/env/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed

In [10]:
rede_neural = MLPClassifier()
print("====== Iniciando busca pelos melhores parâmetros do algoritmo Redes neurais no dataset Teaching ....\n")

param_dist = {'hidden_layer_sizes':list(np.arange(10,20)),'activation':['logistic','identity','tanh'], 'max_iter':[100,200,300,400],'alpha':[0.0001,0.001, 0.01, 0.1], 'solver':['sgd','adam']}

grid_search = GridSearchCV(rede_neural, param_grid=param_dist, cv=kfold_ajuste_parametros, scoring='accuracy', refit=False)
grid_search.fit(teaching_values, teaching_classes)
redes_neurais_param = grid_search.best_params_
print("Redes neurais: %s \n\n" % redes_neurais_param)

====== Iniciando busca pelos melhores parâmetros do algoritmo Redes neurais no dataset Teaching ....

Redes neurais: {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': 16, 'max_iter': 400, 'solver': 'adam'} 




/home/anapedroso/Documentos/IA-desempenho-algoritmos-classificacao/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/anapedroso/Documentos/IA-desempenho-algoritmos-classificacao/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/anapedroso/Documentos/IA-desempenho-algoritmos-classificacao/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/anapedroso/Documentos/IA-desempenho-algoritmos-classificacao/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_pe

In [11]:
kf = model_selection.StratifiedKFold(n_splits = 5)
arvore_decisao = tree.DecisionTreeClassifier(criterion='entropy', splitter='best', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, class_weight=None, presort='deprecated', ccp_alpha=0.0)
vizinhos_proximos = KNeighborsClassifier(n_neighbors=3, weights = 'distance')
naive_bayes_gaussian = GaussianNB()
regressao_logistica = LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, class_weight=None, random_state=None, solver='saga', max_iter=100, multi_class='auto', verbose=0, warm_start=False, n_jobs=None, l1_ratio=None)
rede_neural = MLPClassifier(hidden_layer_sizes=(5,), activation="logistic", max_iter= 300, alpha=0.001, solver="sgd", tol=1e-4, verbose=True, learning_rate_init=.01)

In [12]:
predicted_classes = dict()
predicted_classes['arvore_decisao'] = np.zeros(teaching_classes.shape)
predicted_classes['vizinhos_proximos'] = np.zeros(teaching_classes.shape)
predicted_classes['naive_bayes_gaussian'] = np.zeros(teaching_classes.shape)
predicted_classes['regressao_logistica'] = np.zeros(teaching_classes.shape)
predicted_classes['rede_neural'] = np.zeros(teaching_classes.shape)

arvore_decisao_best_param = DecisionTreeClassifier(**decisionTreeBestParams)
knn_vizinhos_proximos_best_param = KNeighborsClassifier(**knnBestParams)
naive_bayes_gaussian = GaussianNB()
regressao_logistica_best_param = LogisticRegression(**regressao_logistica_param)
rede_neural_best_param = MLPClassifier(**redes_neurais_param)

In [13]:
for train, test in kfold_treinamento.split(teaching_values, teaching_classes):
    data_train, target_train = teaching_values[train], teaching_classes[train]
    data_test, target_test = teaching_values[test], teaching_classes[test]

    arvore_decisao_best_param = arvore_decisao_best_param.fit(data_train, target_train)
    arvore_decisao_predicted = arvore_decisao_best_param.predict(data_test)
    predicted_classes['arvore_decisao'][test] = arvore_decisao_predicted

    knn_vizinhos_proximos_best_param = knn_vizinhos_proximos_best_param.fit(data_train, target_train)
    vizinhos_proximos_predicted = knn_vizinhos_proximos_best_param.predict(data_test)
    predicted_classes['vizinhos_proximos'][test] = vizinhos_proximos_predicted

    naive_bayes_gaussian = naive_bayes_gaussian.fit(data_train, target_train)
    naive_bayes_gaussian_predicted = naive_bayes_gaussian.predict(data_test)
    predicted_classes['naive_bayes_gaussian'][test] = naive_bayes_gaussian_predicted

    regressao_logistica_best_param = regressao_logistica_best_param.fit(data_train, target_train)
    regressao_logistica_predicted = regressao_logistica_best_param.predict(data_test)
    predicted_classes['regressao_logistica'][test] = regressao_logistica_predicted

    rede_neural_best_param = rede_neural_best_param.fit(data_train, target_train)
    rede_neural_predicted = rede_neural_best_param.predict(data_test)
    predicted_classes['rede_neural'][test] = rede_neural_predicted


/home/anapedroso/Documentos/IA-desempenho-algoritmos-classificacao/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (400) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/anapedroso/Documentos/IA-desempenho-algoritmos-classificacao/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (400) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/anapedroso/Documentos/IA-desempenho-algoritmos-classificacao/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (400) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/anapedroso/Documentos/IA-desempenho-algoritmos-classificacao/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_pe

In [14]:
for classificador in predicted_classes.keys():
    print("================================================================================================")
    print("Resultados do classificador %s\n%s\n"
          %(classificador, metrics.classification_report(teaching_classes, predicted_classes[classificador])))
    print("Matriz de confusão: \n%s\n\n\n" % metrics.confusion_matrix(teaching_classes, predicted_classes[classificador]))

Resultados do classificador arvore_decisao
              precision    recall  f1-score   support

           1       1.00      1.00      1.00        49
           2       1.00      1.00      1.00        50
           3       1.00      1.00      1.00        52

    accuracy                           1.00       151
   macro avg       1.00      1.00      1.00       151
weighted avg       1.00      1.00      1.00       151


Matriz de confusão: 
[[49  0  0]
 [ 0 50  0]
 [ 0  0 52]]



Resultados do classificador vizinhos_proximos
              precision    recall  f1-score   support

           1       0.98      1.00      0.99        49
           2       0.96      0.94      0.95        50
           3       0.96      0.96      0.96        52

    accuracy                           0.97       151
   macro avg       0.97      0.97      0.97       151
weighted avg       0.97      0.97      0.97       151


Matriz de confusão: 
[[49  0  0]
 [ 1 47  2]
 [ 0  2 50]]



Resultados do classificad